Snapshot


In [ ]:
import numpy as np
import readgadget
import MAS_library as MASL
import os

def process_and_save_snapshot(snapshot, save_path, grid=512, MAS='CIC', ptype=[1]):
    # Read header
    header = readgadget.header(snapshot)
    BoxSize = header.boxsize / 1e3  # Mpc/h
    Masses = header.massarr * 1e10  # Masses of the particles in Msun/h

    # Read positions of the particles
    pos = readgadget.read_block(snapshot, "POS ", ptype) / 1e3  # positions in Mpc/h
    
    # Create density field
    delta = np.zeros((grid, grid, grid), dtype=np.float32)
    MASL.MA(pos, delta, BoxSize, MAS, verbose=False)
    delta *= Masses[1]

    # Compute mean density for the first 5 slices along the first axis
    mean_density = np.mean(delta[:5, :, :], axis=0)

    # Extract particles in the first 10 Mpc/h region along the x-axis
    indexes = np.where((pos[:, 0] < 10))
    pos_slide = pos[indexes]

    # Ensure the save path exists
    os.makedirs(save_path, exist_ok=True)

    # Save the numpy arrays
    np.save(os.path.join(save_path, 'mean_density.npy'), mean_density)
    np.save(os.path.join(save_path, 'pos_slide.npy'), pos_slide)

# Example usage:
snapshot_path = '/home/jovyan/Data/Snapshots/fiducial/0/snapdir_004/snap_004'
save_path = './data'
process_and_save_snapshot(snapshot_path, save_path)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import readgadget  # Assuming readgadget is a module for reading gadget files
import MAS_library as MASL  # Assuming MAS_library is a module for mass-assignment schemes

def get_image(path, output_path='./2d_images'):
    # Extract the base path
    base_path = '/'.join(path.split('/')[:4])

    add_path = 'snapdir_004/snap_004'
    
    final_path = os.path.join(base_path, add_path)
    snapshot = final_path
    grid = 512    # the density field will have grid^3 voxels
    MAS = 'CIC'   # Mass-assignment scheme: 'NGP', 'CIC', 'TSC', 'PCS'
    verbose = True  # whether to print information about the progress
    ptype = [1]     # [1](CDM), [2](neutrinos) or [1,2](CDM+neutrinos)
    
    header = readgadget.header(snapshot)
    BoxSize = header.boxsize / 1e3  # Mpc/h
    redshift = header.redshift  # redshift of the snapshot
    Masses = header.massarr * 1e10  # Masses of the particles in Msun/h

    # Read positions, velocities, and IDs of the particles
    pos = readgadget.read_block(snapshot, "POS ", ptype) / 1e3  # positions in Mpc/h
    
    delta = np.zeros((grid, grid, grid), dtype=np.float32)
    MASL.MA(pos, delta, BoxSize, MAS, verbose=verbose)
    delta *= Masses[1]
    mean_density = np.mean(delta[:5, :, :], axis=0)  # Take the first 5 components along the first axis and compute the mean value
    
    fig = figure(figsize=(20, 10))
    ax1 = plt.imshow(mean_density.T, cmap='gnuplot', vmin=0.0, vmax=1e13, origin='lower')
    plt.colorbar(ax1)
    
    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)
    
    # Save the generated image
    plt.savefig(os.path.join(output_path, f'{base_path.replace("/", "_")}_image.png'))
    plt.show()

# Example usage
get_image('data/Snapshots/fiducial/12030/snapdir_004')
